# SPL Point Processing 
Load the file 

In [1]:
import pandas as pd
import math

file = "FantaSPL_V2.3.xlsx"
# Load the Excel file
xls = pd.ExcelFile(file)

Load the sheets into dataframes

In [2]:
# Load the 'Games' sheet into a DataFrame
games_df = pd.read_excel(xls, sheet_name='Games')
# Load the 'Parameters' sheet into a DataFrame
parameters_df = pd.read_excel(xls, sheet_name='Parameters')
# Load the 'Points' sheet into a DataFrame
points_df = pd.read_excel(xls, sheet_name='Points')
# Load the 'Players' sheet into a DataFrame
players_df = pd.read_excel(xls, sheet_name='Players')

/home/codespace/.python/current/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Load the parameters as a dictionary

In [3]:
# Convert the DataFrame to a dictionary for easy access
parameters_dict = parameters_df.set_index('Parameter').to_dict()

# Games Summary

Add winning Team and Number of Players

In [4]:
# Add a new column 'Winning Team' based on the goal difference
games_df['Winning Team'] = 'Draw'
games_df.loc[games_df['Team A Goals'] > games_df['Team B Goals'], 'Winning Team'] = 'Team A'
games_df.loc[games_df['Team A Goals'] < games_df['Team B Goals'], 'Winning Team'] = 'Team B'

# Calculate the number of players for each game based on the 'Date' column
players_per_game = points_df.groupby('Date').size()

# Assign a name to the Series
players_per_game.name = 'Number of Players'

# Merge the number of players with the games_df on the 'Date' column
games_df = games_df.merge(players_per_game, left_on='Date', right_index=True, how='left')
games_df = games_df.rename(columns={0: 'Number of Players'})

Adding season and gameweek

In [5]:
# 1. Assign 'Season' column
games_df['Season'] = games_df['Date'].apply(lambda x: 1 if x.month >= 1 and x.month <= 7 and x.year == 2023 else 2)

# 2. Assign 'Gameweek' column
games_df['Gameweek'] = games_df.groupby('Season').cumcount() + 1


Adding Match Type

In [6]:
# Add 'Match Type' column based on the 'Number of Players'
games_df['Match Type'] = '11-a-side'
games_df.loc[games_df['Number of Players'] <= 10, 'Match Type'] = '5-a-side'
games_df.loc[(games_df['Number of Players'] > 10) & (games_df['Number of Players'] < 17), 'Match Type'] = '7-a-side'

Reordering Columns

In [7]:
# Reorder the columns in games_df as specified
column_order = ['Date', 'Season', 'Gameweek', 'Match Type', 'Winning Team', 'Team A Goals', 'Team B Goals', 'Number of Players']
games_df = games_df[column_order]

Saving Games as an excel file

In [8]:
# Save the games_df DataFrame as an Excel file
games_excel = "games.xlsx"
games_df.to_excel(games_excel, index=False)


# Points Calculation

In [9]:
# Merge the 'Points' DataFrame with the 'games_df' DataFrame based on the 'Date' column
points_calculation = points_df.merge(games_df, on='Date', how='left')

# Update the position_dict with the correct column name
position_dict = {
    '5-a-side': players_df.set_index('Player')['Position 5 a-side'].to_dict(),
    '7-a-side': players_df.set_index('Player')['Position 7-a-side'].to_dict(),
    '11-a-side': players_df.set_index('Player')['Position 11-a-side'].to_dict()
}

def get_position(row):
    if pd.notna(row['Game Position']):
        return row['Game Position']
    return position_dict[row['Match Type']].get(row['Player'], 'Unknown')

points_calculation['Position'] = points_calculation.apply(get_position, axis=1)
points_calculation = points_calculation.drop("Game Position", axis=1)


In [10]:

points_calculation['Goal Difference'] = points_calculation.apply(
    lambda row: row['Team A Goals'] - row['Team B Goals'] if row['Team'] == 'Team A' else row['Team B Goals'] - row['Team A Goals'],
    axis=1
)

# Adjust for the winning team
points_calculation['Goal Difference'] = points_calculation.apply(
    lambda row: -abs(row['Goal Difference']) if (row['Winning Team'] != row['Team'] and row['Winning Team'] != 'Draw') else abs(row['Goal Difference']),
    axis=1
)

points_calculation['Game Outcome'] = points_calculation['Goal Difference'].apply(
    lambda x: 'Draw' if x == 0 else ('Win' if x > 0 else 'Loss')
)


In [11]:
def get_game_outcome_points(row):
    match_type = row['Match Type']
    game_outcome = row['Game Outcome']
    return parameters_dict[match_type][game_outcome]

points_calculation['Game Outcome Points'] = points_calculation.apply(get_game_outcome_points, axis=1)

Participation Points

In [12]:
# Add 'Participation' column to the merged_df based on the 'Match Type' column and the parameters_dict
points_calculation['Participation'] = points_calculation['Match Type'].apply(lambda x: parameters_dict[x]['Participation'])


In [13]:
# Add 'Goal Points' column to the merged_df based on the 'Goals', 'Match Type' columns, and the parameters_dict
points_calculation['Goal Points'] = points_calculation.apply(lambda row: row['Goals'] * parameters_dict[row['Match Type']]['Goal'], axis=1)

In [14]:
# Add 'Own Goal Points' column to the merged_df based on the 'Own Goals', 'Match Type' columns, and the parameters_dict
points_calculation['Own Goal Points'] = points_calculation.apply(lambda row: row['Own Goals'] * parameters_dict[row['Match Type']]['Own Goal'], axis=1)

In [15]:
# Add 'SPL Bonus Points' 
points_calculation['SPL Bonus Points'] = points_calculation.apply(lambda row: row['SPL Bonus'] * parameters_dict[row['Match Type']]['SPL Bonus'], axis=1)

In [ ]:
# Add 'MVP Points'
points_calculation['MVP Points'] = points_calculation.apply(lambda row: row['MVP'] * parameters_dict[row['Match Type']]['MVP'], axis=1)

In [ ]:
# Add 'Friend Points'
points_calculation['Friend Referrals Points'] = points_calculation.apply(lambda row: row['Friend Referrals'] * parameters_dict[row['Match Type']]['Friend Referrals'], axis=1)

In [ ]:
# Add 'Penalty Points'
points_calculation['Penalty Points'] = points_calculation.apply(lambda row: row['Penalty'] * parameters_dict[row['Match Type']]['Penalty'], axis=1)

In [ ]:
# Add 'Goalkeeper Points' column to the merged_df based on the 'Position' and 'Match Type' columns, and the parameters_dict
points_calculation['Goalkeeper Points'] = points_calculation.apply(
    lambda row: parameters_dict[row['Match Type']]['Goalkeeper Score'] if row['Position'] == 'Goalkeeper' else 0,
    axis=1
)


In [ ]:
# Add 'Goals Conceded' column to the merged_df based on the 'Team', 'Team A Goals', and 'Team B Goals' columns
points_calculation['Goals Conceded'] = points_calculation.apply(
    lambda row: row['Team B Goals'] if row['Team'] == 'Team A' else row['Team A Goals'],
    axis=1
)

In [ ]:
# Modify the function to round up negative 'Defensive Score Points' to 0
def calculate_defensive_score(row):
    base_score = parameters_dict[row['Match Type']]['Defensive Score'] - row['Goals Conceded']
    
    if row['Position'] in ['Goalkeeper', 'Defender', 'Defensive']:
        return max(base_score, 0)
    elif row['Position'] in ['Midfielder', 'Outfield']:
        return max(math.ceil(base_score / 2), 0)
    elif row['Position'] in ['Forward', 'Offensive']:
        return 0
    else:
        return 'Unknown'

# Recalculate 'Defensive Score Points' column with the modified function
points_calculation['Defensive Score Points'] = points_calculation.apply(calculate_defensive_score, axis=1)


In [ ]:
# Modify the function to calculate 'Midfield Score' based on 'Goal Difference'
def calculate_midfield_score(row):
    if row['Position'] in ['Midfielder', 'Offensive', 'Outfield']:
        return max(row['Goal Difference'], 0)
    elif row['Position'] == 'Forward':
        return max(math.ceil(row['Goal Difference'] / 2), 0)
    else:
        return 0

# Recalculate 'Midfield Score' column with the modified function
points_calculation['Midfield Score'] = points_calculation.apply(calculate_midfield_score, axis=1)



In [ ]:
# List of columns to sum
columns_to_sum = [
    'Participation', 'Goal Points', 'Own Goal Points', 'SPL Bonus Points', 
    'MVP Points', 'Friend Referrals Points', 'Goalkeeper Points', 'Defensive Score Points', 'Midfield Score', 'Penalty Points','Game Outcome Points'
]

# Add a new column 'Total Points' which is the sum of the specified columns
points_calculation['Total Points'] = points_calculation[columns_to_sum].sum(axis=1)


In [ ]:
# Reorder the columns in merged_df as specified
column_order = [
    'Date', 'Season', 'Gameweek',  'Match Type',  'Player', 'Position', 'Team', 'Winning Team','Game Outcome', 'Number of Players', 'Goals', 
    'Own Goals', 'SPL Bonus', 'MVP', 'Friend Referrals', 'Penalty', 'Team A Goals', 'Team B Goals', 
    'Goals Conceded', 'Goal Difference', 'Participation','Game Outcome Points','Penalty Points', 'Goal Points', 'Own Goal Points', 'SPL Bonus Points', 
    'MVP Points', 'Friend Referrals Points', 'Goalkeeper Points', 'Defensive Score Points', 'Midfield Score', 'Total Points'
]
points_calculation = points_calculation[column_order]



In [ ]:
# Save the games_df DataFrame as an Excel file
points_excel = "points.xlsx"
points_calculation.to_excel(points_excel, index=False)